In [ ]:
# testing BayLIME on images
# based on https://github.com/x-y-zhao/BayLime/blob/master/BayLIME_tutorial_images.ipynb
# Tatsuo Okubo
# 2021/11/06

In [2]:
from BayLime.lime import lime_image

In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_ubyte
import tensorflow as tf
from tensorflow import keras

In [4]:
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

In [5]:
tf.__version__

'2.4.1'

In [ ]:
keras.__version__

In [ ]:
inet_model = tf.keras.applications.InceptionV3(weights="imagenet")  # load weights from a model pre-trained on ImageNet

In [ ]:
def transform_img_fn(path_list):
    out = []
    for img_path in path_list:
        img = image.load_img(img_path, target_size=(299, 299))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        out.append(x)
    return np.vstack(out)

In [ ]:
images = transform_img_fn([os.path.join('..', 'BayLime', 'data','dogs.jpg')])
plt.imshow(images[0] / 2 + 0.5)
plt.show()
preds = inet_model.predict(images)
for x in decode_predictions(preds)[0]:
    print(x)

In [ ]:
images = transform_img_fn([os.path.join('..', 'BayLime', 'data','5.jpg')])
plt.imshow(images[0] / 2 + 0.5)
plt.show()
preds = inet_model.predict(images)
for x in decode_predictions(preds)[0]:
    print(x)

In [ ]:
images = transform_img_fn([os.path.join('..', 'BayLime', 'data','throne.jpg')])
plt.imshow(images[0] / 2 + 0.5)
plt.show()
preds = inet_model.predict(images)
for x in decode_predictions(preds)[0]:
    print(x)

In [ ]:
images = transform_img_fn([os.path.join('..', 'BayLime', 'data','eagle_sheep.jpg')])
plt.imshow(images[0] / 2 + 0.5)
plt.show()
preds = inet_model.predict(images)
for x in decode_predictions(preds)[0]:
    print(x)

In [ ]:
preds[0].shape

In [ ]:
# make sure to convert float to double
image_uint = img_as_ubyte(images)
image_uint.dtype

In [ ]:
def predict_fn(images):
    return session.run(probabilities, feed_dict={processed_images: images})

In [ ]:
explainer = lime_image.LimeImageExplainer(feature_selection='none')#kernel_width=0.1
explanation = explainer.explain_instance(image_uint[0], inet_model.predict,
                                         top_labels=3, hide_color=0, batch_size=10,
                                         num_samples=200, model_regressor='non_Bay')

In [ ]:
# original LIME
from lime import lime_image
explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(image_uint, predict_fn, top_labels=5, hide_color=0, num_samples=1000)